# Assumptions

- Safe objects will persist in SCN memory until they are either moved, deleted manually or the SCN is powered off. 
- A LinearRegressionClient object will be on each SCN
- A LinearRegressionFederated object will be with the orchestrator
- 

In [1]:
import torch
from torch.autograd import Variable
from torch.nn import functional as F

class LinearRegressionClient(torch.nn.Module):
    def __init__(self):
        super(LinearRegressionClient, self).__init__()
        # self.linear = torch.nn.Linear(in_layer, out_layer, bias=True)
        # self.criterion = criterion
        # optimizer = torch.optim.SGD(self.linear.parameters(), lr=learn_rate)
        # self.optimizer = optimizer
    
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

    def train_model(self, epochs: int, x_data, y_data):
        for epoch in range(epochs):
            model = self.linear
            model.train()
            self.optimizer.zero_grad()
            # Forward pass
            y_pred = model(x_data)
            # Compute Loss
            loss = self.criterion(y_pred, y_data)
            # Backward pass
            loss.backward()
            self.optimizer.step()
        print("Loss: "+ str(loss.data.view(-1)))

    
    def set_parameters_as_tensor(self, weights: torch.Tensor):
        if not isinstance(weights, torch.Tensor):
            print("Converting weights to tensor type")
            weights = torch.tensor(weights)
        current_index = 0 
        for parameter in self.parameters():
            numel = parameter.data.numel()
            size = parameter.data.size()
            parameter.data.copy_(
                            weights[current_index:current_index +
                                                numel].view(size))
            current_index+=1

    def get_parameters_as_tensor(self):
        params = [param.data.view(-1) for param in self.parameters()]
        params = torch.cat(params)
        params = params.cpu()
        return params

    def run(in_layer, out_layer, x_data, y_data learn_rate=0.1, criterion=None, optimizer=None, epochs=10):
        
        super(LinearRegressionClient, self).__init__()
        linear = torch.nn.Linear(in_layer, out_layer, bias=True)
        criterion = torch.nn.MSELoss(size_average=False)
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=learn_rate)

        train_model(epochs, x_data, y_data)






SyntaxError: invalid syntax (340585904.py, line 51)

In [331]:
class LinearRegressionFederated:
    def __init__(self, models: LinearRegressionClient, data_federation):
        self.models = models
        self.data_federation = data_federation
    
    def get_average_model(self):

        #Pull parameters for each model locally
        model_parameters = []
        for model in self.models:
            model_parameters.append(model.get_parameters_as_tensor())
        
        #TODO: Push list of model parameters to some SCN
        for param in model_parameters:
            # param.send(models[random])
            break
            
        #Calculate the Mean Remotely
        #TODO: DO this using existing safe object
        model_sum = 0.
        for param in model_parameters:
            model_sum += param
        model_avg = model_sum/len(self.models)
        #TODO: Pull result to orchestrator

        return model_avg
    
    def train_cycle(self, epochs: int):
        #train individual models on data
        for i in range(len(self.models)):
            models[i].train_model(epochs, data_federation[i][0], data_federation[i][1])
        
        #average the models
        avg_model = self.get_average_model()
        for i in range(len(self.models)):
            models[i].set_parameters_as_tensor(avg_model)

        #TODO:Pull Result locally           


In [332]:
#Data Starts with respective owners A and B
x_dataA = Variable(torch.Tensor([[10.0], [9.0], [3.0], [2.0]]))
y_dataA = Variable(torch.Tensor([[90.0], [80.0], [50.0], [30.0]]))
x_dataB = Variable(torch.Tensor([[7.0], [8.0], [3.0], [1.0]]))
y_dataB = Variable(torch.Tensor([[60.0], [70.0], [40.0], [20.0]]))
data_federation = [[x_dataA, y_dataA], [x_dataB, y_dataB]]
        
modelA = LinearRegressionClient(1,1)
modelB = LinearRegressionClient(1,1)
models = [modelA, modelB]

federated_model = LinearRegressionFederated(models, data_federation)

In [329]:
for i in range(5):
    # print(federated_model.models[0].get_parameters_as_tensor())
    # print(federated_model.models[1].get_parameters_as_tensor())
    federated_model.train_cycle(2)
    # print(federated_model.models[0].get_parameters_as_tensor())
    # print(federated_model.models[1].get_parameters_as_tensor())

Loss: tensor([31640224.])
Loss: tensor([6174763.])
Loss: tensor([3.1239e+13])
Loss: tensor([7.9251e+12])
Loss: tensor([3.3114e+19])
Loss: tensor([8.4019e+18])
Loss: tensor([3.5102e+25])
Loss: tensor([8.9065e+24])
Loss: tensor([3.7210e+31])
Loss: tensor([9.4414e+30])
